In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 이동 평균을 계산하는 함수
def calculate_moving_average(data, window_size=3):
    return data.rolling(window=window_size).mean()

In [3]:
# 수익 함수의 기울기 계산 함수 (주어진 코드)
def revenue_gradient(P: float, C: float, E: float) -> float:
    return C * (E + 1) * P**E

# 경사 상승법 (Gradient Ascent) 함수 (주어진 코드)
def gradient_ascent(C: float, E: float, initial_price: float, learning_rate: float, iterations: int, rewatch_prob: float) -> float:
    P = initial_price  # 초기 가격 설정, 정가
    for i in range(iterations):
        grad = revenue_gradient(P, C, E)  # 기울기 계산
        P = P + learning_rate * grad  # 가격 업데이트
        if abs(weighted_grad) < 1e-6:  # 가중된 기울기가 매우 작으면 종료
            break
    return P

In [4]:
# 특정 공연코드에 대해 시간별로 클러스터별 재관람 확률을 반영한 최적의 가격 계산
def calculate_optimal_prices(data, C, initial_price, learning_rate, iterations, cluster_rewatch_probs):
    optimal_prices = []
    for _, row in data.iterrows():
        E = - row['예측할인율']  # 예측된 할인율 사용
        cluster_id = row['cluster']  # 클러스터 ID 가져오기
        rewatch_prob = cluster_rewatch_probs[cluster_id]  # 해당 클러스터의 재관람 확률 가져오기
        optimal_price = gradient_ascent(C, E, initial_price, learning_rate, iterations, rewatch_prob)
        optimal_prices.append(optimal_price)
    return optimal_prices

In [5]:
# 클러스터별 최적 가격을 가중치로 조정하여 최종 가격을 결정하는 함수
def calculate_weighted_final_price(data, C, initial_price, learning_rate, iterations, cluster_rewatch_probs):
    weighted_prices_sum = 0
    total_rewatch_prob = 0

    # 각 클러스터별 최적 가격 계산 및 가중치 반영
    for cluster_id in data['cluster'].unique():
        cluster_data = data[data['cluster'] == cluster_id]
        optimal_prices = calculate_optimal_prices(cluster_data, C, initial_price, learning_rate, iterations, cluster_rewatch_probs)

        # 가중치로 사용할 재관람 확률
        rewatch_prob = cluster_rewatch_probs[cluster_id]

        # 가중치를 반영한 가격 계산
        weighted_prices_sum += sum(optimal_prices) * rewatch_prob
        total_rewatch_prob += rewatch_prob

    # 최종 가격은 가중된 가격들의 합을 재관람 확률의 총합으로 나눈 값
    final_price = weighted_prices_sum / total_rewatch_prob if total_rewatch_prob > 0 else initial_price

    return final_price

In [6]:
# 클러스터별 최적 가격 계산 및 시각화 함수
def visualize_and_save_optimal_prices(data, C, initial_price, learning_rate, iterations, cluster_rewatch_probs, output_csv):
    all_optimal_prices = []

    # 클러스터별로 처리
    for cluster_id in data['cluster'].unique():
        cluster_data = data[data['cluster'] == cluster_id]

        # 최적 가격 계산
        optimal_prices = calculate_optimal_prices(cluster_data, C, initial_price, learning_rate, iterations, cluster_rewatch_probs)

        # 시각화
        plt.plot(cluster_data['공연일시'], optimal_prices, label=f'Cluster {cluster_id}')

        # 데이터를 데이터프레임에 저장
        cluster_data['최적가격'] = optimal_prices
        all_optimal_prices.append(cluster_data[['공연코드', '공연일시', 'cluster', '최적가격']])

    # 가중치 적용한 최종 가격 계산
    final_price = calculate_weighted_final_price(data, C, initial_price, learning_rate, iterations, cluster_rewatch_probs)
    print(f"Weighted Final Optimal Price: {final_price}")

    # 그래프 설정 및 표시
    plt.xlabel('공연일시')
    plt.ylabel('최적 가격')
    plt.title('클러스터별 시간에 따른 최적 가격')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 모든 클러스터 데이터를 하나로 합치고 CSV로 저장
    final_df = pd.concat(all_optimal_prices)
    final_df['최종최적가격'] = final_price
    final_df.to_csv(output_csv, index=False)

In [7]:
# 모든 클러스터에 대해 최적의 가격을 계산하고 엑셀 파일로 저장하는 함수
def save_optimal_prices_to_excel(data, initial_price, learning_rate, iterations, cluster_rewatch_probs, file_name):
    # 최적 가격 결과를 담을 데이터프레임 생성
    result_df = pd.DataFrame()

    for cluster_id in data['cluster'].unique():
        cluster_data = data[data['cluster'] == cluster_id].copy()
        cluster_data['최적가격'] = calculate_optimal_prices(cluster_data, initial_price, learning_rate, iterations, cluster_rewatch_probs)

        # 결과 데이터프레임에 추가
        result_df = pd.concat([result_df, cluster_data[['공연코드', '공연일시', 'cluster', '최적가격']]])

    # 가중치 적용한 최종 가격 계산
    final_price = calculate_weighted_final_price(data, initial_price, learning_rate, iterations, cluster_rewatch_probs)

    # 최종 가격을 추가하여 엑셀 파일로 저장
    result_df['최종최적가격'] = final_price
    result_df.to_excel(file_name, index=False)
    print(f"페르소나 별 최적 가격을 {file_name} 파일로 저장했습니다. Weighted Final Price: {final_price}")